# Generative Adversarial Network #2 (DCGAN)
Reference: [Understanding and building Adversarial Networks (GANs) - Deep Learning with PyTorch](https://becominghuman.ai/understanding-and-building-generative-adversarial-networks-gans-8de7c1dc0e25)

![GAN Concept](https://cdn-images-1.medium.com/max/1600/1*YH3b1fARO-bf6gU3kyzT4A.jpeg)

In [1]:
#importing required libraries
from __future__ import print_function
import time
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
# Setting hyperparameters
batchSize = 64 
imageSize = 64    # Size of images generated: (64 x 64)

In [3]:
# Creating the transformations
transform = transforms.Compose([transforms.Resize(imageSize),    # transforms.Scale will be deprecated.
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])

In [4]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2)

Files already downloaded and verified


In [5]:
# defining a universal function to initialise the weights
def weights_init(m):    # 'm' is a neural network
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

### Outline of training process

![Training Process](https://cdn-images-1.medium.com/max/1600/1*NFO8IogPJRf_eGKBZnd-Fg.png)

In [6]:
# Generator class
class G(nn.Module):
    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )
    
    def forward(self, input):
        output = self.main(input)    # Input will be random vector of size 100
        return output

In [7]:
# Create Generator
netG = G() 
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [8]:
class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),    # Output will be between 0 and 1
            nn.Sigmoid()
        )
    
    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

In [9]:
# Create Discriminator
netD = D() 
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [10]:
# Error prediction measurement
criterion = nn.BCELoss()    # BCE: Binary Cross Entropy
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

In [ ]:
# Training Start
# 25 Epochs

for epoch in range(25):    
    
    # Iterate over images within the dataset
    for i, data in enumerate(dataloader, 0):
        start = time.time()
        
        ### TRAIN DISCRIMINATOR ###
        # Initialize discriminator weights
        netD.zero_grad()
        
        # Train the discriminator with the real image dataset first.
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        # Now, train with fake images, generated by Generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        # Back-propagate the error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()
        
        
        ### TRAIN GENERATOR ###
        # Update generator weights
        netG.zero_grad()    # Initialize weights
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        
        ## Print the losses and save the real images & the generated images for every 100 steps.
        if i % 250 == 0:
            end = time.time()
            print('[%d/%d][%d/%d]    Loss_D: %-8.4f    Loss_G: %-8.4f    Time Cost: %-6.3fsec' \
                  % (epoch, 25, i, len(dataloader), errD.item(), errG.item(), end - start))    # .data[0] will be deprecated.
            start = time.time()
            torch.save(netD.state_dict(), "./models/Discriminator/D_{}_{}.pth".format(epoch, i))
            torch.save(netG.state_dict(), "./models/Generator/G_{}_{}.pth".format(epoch, i))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

[0/25][0/782]    Loss_D: 2.1004      Loss_G: 2.4990      Time Cost: 3.898 sec
[0/25][250/782]    Loss_D: 0.1520      Loss_G: 5.1958      Time Cost: 3.339 sec
[0/25][500/782]    Loss_D: 0.5522      Loss_G: 3.5930      Time Cost: 3.344 sec
[0/25][750/782]    Loss_D: 0.4159      Loss_G: 3.2064      Time Cost: 3.795 sec
[1/25][0/782]    Loss_D: 0.6473      Loss_G: 6.4075      Time Cost: 3.768 sec
[1/25][250/782]    Loss_D: 0.3442      Loss_G: 4.0436      Time Cost: 3.582 sec
